# Electrolyzer Simulator — Figuras consolidadas (v3)

Este notebook gera **4 gráficos** usados no artigo (6 páginas):
1) Curvas de polarização (I–V) — AWE/PEM/SOEC  
2) Eficiência energética × Temperatura — AWE/PEM (SOEC como referência)  
3) Distribuição de sobretensões × Temperatura — AWE  
4) Dinâmica (degraus de corrente) — AWE (1º ordem **e** modelo de balanço de energia)

**Modelagem didática**: Nernst, Tafel, queda ôhmica (ASR(T)), concentração, eficiência faradaica simples.  
Parâmetros foram ajustados para refletir **tendências da literatura**: aumento de eficiência com T até um **ótimo** (~70–75 °C) em AWE/PEM e melhor desempenho em alta T em SOEC.

> Use as células de *Calibração* para bater números-alvo (ex.: ~82,4% @ 75 °C em AWE; ~79,6% @ 70 °C em PEM).


In [ ]:
# Imports e constantes
import numpy as np
import matplotlib.pyplot as plt

# Regras do artigo: matplotlib puro, sem seaborn; uma figura por gráfico.
plt.rcParams.update({
    "figure.dpi": 140,
    "font.size": 12,
    "axes.grid": True
})

R = 8.314462618  # J/mol/K
F = 96485.33212  # C/mol


In [ ]:
# Núcleo eletroquímico
def nernst_voltage(T_K, p_H2=1.0, p_O2=1.0, p_H2O=1.0):
    # 1.229 V @ 25°C; correção ~0,9 mV/K + termo log(p)
    V0 = 1.229
    V_T = V0 - 0.0009*(T_K - 298.15)
    return V_T + (R*T_K)/(2*F)*np.log((p_H2*(p_O2**0.5))/max(p_H2O, 1e-12))

def tafel_eta(i, i0, T_K, alpha=0.5):
    i = np.maximum(i, 1e-12)
    return (R*T_K)/(alpha*F)*np.log(i/np.maximum(i0,1e-20))

def ohmic_eta(i, ASR):
    return i*ASR

def concentration_eta(i, i_lim, T_K):
    eps = 1e-12
    ratio = np.clip(1 - i/np.maximum(i_lim, eps), eps, 1-1e-9)
    return (R*T_K)/(2*F)*np.log(1/ratio)

def v_cell(i, params, T_K):
    Vrev = nernst_voltage(T_K, params.get("p_H2",1), params.get("p_O2",1), params.get("p_H2O",1))
    eta_act = tafel_eta(i, params["i0"](T_K), T_K, params.get("alpha",0.5))
    eta_ohm = ohmic_eta(i, params["ASR"](T_K))
    eta_conc = concentration_eta(i, params["i_lim"](T_K), T_K)
    return Vrev + eta_act + eta_ohm + eta_conc, (Vrev, eta_act, eta_ohm, eta_conc)

def faradaic_efficiency(i):
    # ~constante ~0.98 para correntes típicas; um pouco menor em i muito baixo
    return 0.985 - 0.02*np.exp(-i/0.2)

def energy_efficiency(Vrev, Vcell, eta_F=0.985):
    return (Vrev/np.maximum(Vcell, 1e-12))*eta_F


## Parametrizações dependentes de temperatura (com ótimo para AWE/PEM)

In [ ]:
def arrhenius(val_25C, Ea_kJmol, T_K):
    Ea = Ea_kJmol*1000.0
    return val_25C*np.exp(-Ea/R*(1/T_K - 1/298.15))

# AWE: perdas caem com T até ~75°C; acima disso, penalidade (evaporação/degradação)
def ASR_AWE(T_K):
    base = 0.24 - 0.0012*(T_K-298.15)            # ↓ com T
    penalty = 0.0008*max(T_K-348.15, 0.0)        # ↑ se T>75°C
    return float(np.clip(base + penalty, 0.06, 0.30))

def i0_AWE(T_K):
    return float(arrhenius(2e-6, 35.0, T_K))     # ganho cinético mais forte

def i_lim_AWE(T_K):
    return float(2.6 + 0.002*(T_K-298.15))

# PEM: melhora até ~70°C; depois penalidade (gestão de membrana/água)
def ASR_PEM(T_K):
    base = 0.20 - 0.0010*(T_K-298.15)
    penalty = 0.0010*max(T_K-343.15, 0.0)        # penaliza >70°C
    return float(np.clip(base + penalty, 0.05, 0.25))

def i0_PEM(T_K):
    return float(arrhenius(1e-5, 28.0, T_K))

def i_lim_PEM(T_K):
    return float(2.4 + 0.0015*(T_K-298.15))

# SOEC: muito baixa ASR em alta T; i0 alto
def ASR_SOEC(T_K):
    # mínimo próximo de 850–900°C; forma suave
    return float(0.02 + 0.06*np.exp(-(T_K-873.15)/220.0))

def i0_SOEC(T_K):
    return float(arrhenius(5e-4, 12.0, T_K))

def i_lim_SOEC(T_K):
    return float(3.0 + 0.002*(T_K-298.15))

PARAMS = {
    "AWE": {"ASR": ASR_AWE, "i0": i0_AWE, "i_lim": i_lim_AWE, "alpha": 0.5, "p_H2":1, "p_O2":1, "p_H2O":1},
    "PEM": {"ASR": ASR_PEM, "i0": i0_PEM, "i_lim": i_lim_PEM, "alpha": 0.5, "p_H2":1, "p_O2":1, "p_H2O":1},
    "SOEC":{"ASR": ASR_SOEC,"i0": i0_SOEC,"i_lim": i_lim_SOEC,"alpha": 0.5, "p_H2":1, "p_O2":1, "p_H2O":1},
}


## Calibração (opcional) para bater eficiência-alvo em um ponto (T*, i*)

In [ ]:
def make_scaled(param, a_scale, i0_scale):
    def ASR_scaled(T): return a_scale*param["ASR"](T)
    def i0_scaled(T):  return i0_scale*param["i0"](T)
    return {"ASR": ASR_scaled, "i0": i0_scaled, "i_lim": param["i_lim"],
            "alpha": param["alpha"], "p_H2":1, "p_O2":1, "p_H2O":1}

def calibrate_params(target_eta=0.80, T_star=333.15, i_star=1.2, param=None, max_iter=24):
    a_s, i0_s = 1.0, 1.0
    for _ in range(max_iter):
        trial = make_scaled(param, a_s, i0_s)
        Vcell,(Vrev, *_)= v_cell(i_star, trial, T_star)
        eta = energy_efficiency(Vrev, Vcell, eta_F=0.985)
        if eta < target_eta:
            a_s *= 0.92     # reduz resistência
            i0_s *= 1.6     # aumenta i0
        else:
            a_s *= 1.02
            i0_s *= 0.90
    tuned = make_scaled(param, a_s, i0_s)
    Vcell,(Vrev, *_)= v_cell(i_star, tuned, T_star)
    eta = energy_efficiency(Vrev, Vcell, eta_F=0.985)
    print(f"Eficiência calibrada @ {T_star-273.15:.1f} °C, i={i_star:.2f} A/cm²: {eta*100:.1f}%")
    print(f"Escalas -> ASR: {a_s:.3f}  |  i0: {i0_s:.3e}")
    return tuned

# Ative/desative conforme necessidade
USE_CAL_AWE = True
USE_CAL_PEM = True

PARAMS_USE = dict(PARAMS)
if USE_CAL_AWE:
    PARAMS_USE["AWE"] = calibrate_params(target_eta=0.824, T_star=348.15, i_star=1.2, param=PARAMS["AWE"]) # ~82,4% @ 75°C
if USE_CAL_PEM:
    PARAMS_USE["PEM"] = calibrate_params(target_eta=0.796, T_star=343.15, i_star=1.2, param=PARAMS["PEM"]) # ~79,6% @ 70°C


## 1) Curvas de polarização (I–V) — AWE, PEM, SOEC

In [ ]:
T_C = 60.0
T_K = 273.15 + T_C
i_grid = np.linspace(0.2, 2.5, 180)

fig, ax = plt.subplots(figsize=(6.5,4.2))
for tech,label in [("AWE","Alkaline"), ("PEM","PEM"), ("SOEC","SOEC")]:
    Vcell,_ = v_cell(i_grid, PARAMS_USE[tech], T_K)
    ax.plot(i_grid, Vcell, label=f"{label} eletrolizer")
ax.set_xlabel("Current density (A/cm$^2$)")
ax.set_ylabel("Cell Voltage (V)")
ax.legend()
plt.tight_layout()
plt.show()


**Descrição:** As curvas I–V exibem o crescimento típico de tensão com a corrente por soma de perdas de ativação, ôhmica e de concentração. Para a mesma densidade de corrente, **SOEC < PEM < AWE** em tensão, refletindo cinética e condução iônica mais favoráveis em alta T.  
**Referências sugeridas:** Zeng & Zhang (2010); Carmo *et al.* (2013); Laguna‑Bercero (2012).

## 2) Eficiência energética × Temperatura — AWE/PEM (SOEC como referência)

In [ ]:
# Faixa 35–80°C para AWE/PEM; SOEC mostrado como referência a 850°C (valor constante)
T_list = np.linspace(308.15, 353.15, 11)  # 35–80 ºC
i_op = 1.2
eff = {"AWE":[], "PEM":[], "SOEC":[]}

for T_K in T_list:
    for tech in ["AWE","PEM"]:
        Vcell,(Vrev, *_)= v_cell(i_op, PARAMS_USE[tech], T_K)
        eff[tech].append(100*energy_efficiency(Vrev, Vcell, eta_F=faradaic_efficiency(i_op)))

# SOEC @ 850°C como linha de referência
T_SOEC = 1123.15
Vcell,(Vrev, *_)= v_cell(i_op, PARAMS_USE["SOEC"], T_SOEC)
eta_SOEC = 100*energy_efficiency(Vrev, Vcell, eta_F=faradaic_efficiency(i_op))
eff["SOEC"] = [eta_SOEC for _ in T_list]

fig, ax = plt.subplots(figsize=(6.5,4.2))
ax.plot(T_list-273.15, eff["AWE"], marker="o", label="Alkaline electrolyzer")
ax.plot(T_list-273.15, eff["PEM"], marker="o", label="PEM electrolyzer")
ax.plot(T_list-273.15, eff["SOEC"], linestyle="--", label="SOEC (ref. @ 850°C)")
ax.set_xlabel("Temperatura (°C)")
ax.set_ylabel("Eficiência Energética (%)")
ax.legend()
plt.tight_layout()
plt.show()


**Descrição:** Para **AWE** e **PEM**, a eficiência **aumenta** com a temperatura até um **ótimo** (~70–75 °C), quando perdas de ativação e ôhmicas são minimizadas; acima disso tende a estabilizar/ceder por limites de materiais e gestão de água. **SOEC** opera com eficiências elevadas em **alta temperatura**; a linha pontilhada indica um patamar típico calculado a 850 °C.  
**Referências sugeridas:** Zeng & Zhang (2010); Carmo *et al.* (2013); Bi *et al.* (2025); Azuan *et al.* (2019); Laguna‑Bercero (2012).

## 3) Distribuição de sobretensões × Temperatura — AWE

In [ ]:
T_list = np.linspace(298.15, 353.15, 12)  # 25–80 ºC
i_op = 1.5
fracs = []
for T_K in T_list:
    Vcell,(Vrev, eta_act, eta_ohm, eta_conc) = v_cell(i_op, PARAMS_USE["AWE"], T_K)
    total_losses = max(eta_act + eta_ohm + eta_conc, 1e-9)
    fracs.append([100*eta_act/total_losses, 100*eta_ohm/total_losses, 100*eta_conc/total_losses])
fracs = np.array(fracs)

fig, ax = plt.subplots(figsize=(6.5,4.2))
ax.stackplot(T_list-273.15, fracs[:,0], fracs[:,1], fracs[:,2], labels=["Ativação","Ôhmica","Concentração"])
ax.set_xlabel("Temperatura (°C)")
ax.set_ylabel("Distribuição de Sobretensão (%)")
ax.legend(loc="upper right")
plt.tight_layout()
plt.show()


**Descrição:** A fração de **ativação** **diminui** com a temperatura (i₀↑), e a **ôhmica** também reduz (melhor condutividade). Em correntes elevadas, a perda de **concentração** tende a crescer relativamente. Essa decomposição explica a melhoria de tensão até a faixa ótima de operação.  
**Referências sugeridas:** Zeng & Zhang (2010); Carmo *et al.* (2013).

## 4) Simulação dinâmica (degraus de corrente) — AWE (1º ordem e balanço de energia)

In [ ]:
# Série de degraus de corrente (A/cm²)
t = np.arange(0, 60*60+1, 10)        # 0–60 min em 10 s
steps_Acm2 = [0.7, 0.6, 1.0, 0.9, 1.2, 1.1, 1.4]
t_step = np.linspace(5, 55, len(steps_Acm2))*60  # s
i_series = np.ones_like(t, dtype=float)*steps_Acm2[0]
for k, tk in enumerate(t_step[1:], start=1):
    i_series[t>=tk] = steps_Acm2[k]

# Modelo 1: 1ª ordem em T (proxy)
T_base = 333.15  # 60 ºC
tau = 15*60
T1 = np.zeros_like(t, dtype=float); T1[0] = T_base
for k in range(1, len(t)):
    T_target = T_base + 4.0*(i_series[k]-np.mean(steps_Acm2))
    T1[k] = T1[k-1] + ((T_target - T1[k-1]) * (t[k]-t[k-1]) / tau)

# Modelo 2: balanço de energia
T_env = 298.15    # 25 ºC
h = 0.6           # W/cm²/K (didático)
C_th = 12.0       # J/cm²/K (didático)
T2 = np.zeros_like(t, dtype=float); T2[0] = T_base
for k in range(1, len(t)):
    Vcell,(Vrev, *_)= v_cell(max(i_series[k-1],1e-6), PARAMS_USE["AWE"], T2[k-1])
    P_loss = max(i_series[k-1]*(Vcell - Vrev), 0.0)  # W/cm²
    dTdt = (P_loss - h*(T2[k-1]-T_env)) / C_th
    T2[k] = T2[k-1] + dTdt*(t[k]-t[k-1])

def efficiency_series(t, i_series, T_series, param):
    eff = np.zeros_like(t, dtype=float)
    for k, (Ti, ik) in enumerate(zip(T_series, i_series)):
        Vcell,(Vrev, *_)= v_cell(max(ik,1e-6), param, Ti)
        eff[k] = 100*energy_efficiency(Vrev, Vcell, eta_F=0.985)
    return eff

eff1 = efficiency_series(t, i_series, T1, PARAMS_USE["AWE"])
eff2 = efficiency_series(t, i_series, T2, PARAMS_USE["AWE"])

fig, ax1 = plt.subplots(figsize=(7.0,4.2))
ax1.plot(t/60.0, eff1, label="Eficiência (%) — 1ª ordem")
ax1.plot(t/60.0, eff2, label="Eficiência (%) — balanço de energia")
ax1.set_xlabel("Tempo (min)")
ax1.set_ylabel("Eficiência Energética (%)")

ax2 = ax1.twinx()
ax2.plot(t/60.0, i_series, ls="--", drawstyle="steps-post", label="Densidade de Corrente (A/cm²)")
ax2.set_ylabel("Densidade de Corrente (A/cm²)")

# Legenda combinada
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines+lines2, labels+labels2, loc="lower left")
plt.tight_layout()
plt.show()

print(f"Média de eficiência (1ª ordem): {eff1.mean():.1f}% | min: {eff1.min():.1f}% | max: {eff1.max():.1f}%")
print(f"Média de eficiência (balanço): {eff2.mean():.1f}% | min: {eff2.min():.1f}% | max: {eff2.max():.1f}%")


**Descrição:** A resposta a degraus de corrente evidencia o acoplamento **térmico-eletroquímico**: variações de carga alteram as perdas e a temperatura interna com uma constante de tempo das dezenas de minutos em AWE. O modelo de balanço de energia reproduz atraso e amortecimento realistas.  
**Referência sugerida:** Carmo *et al.* (2013).